In [12]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)
ALL_IN_ONE_WORKER = False

In [13]:
@dataclass
class Message:
    content: str

In [14]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address = "localhost:50051")
host.start()

In [15]:
serper = GoogleSerperAPIWrapper()
langchain_serper = Tool(name = "internet_search", func = serper.run, description="Useful when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [16]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the cons of AutoGen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [17]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

In [18]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")

In [19]:
response = await worker.send_message(Message(content="Go"), agent_id)

In [20]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some reasons in favor of choosing AutoGen for your new AI Agent project:

1. **Collaboration and Efficiency**: AutoGen enables agents to communicate through conversation, which facilitates collaboration and divergent thinking. This ability leads to improved problem-solving and deeper reasoning among agents.

2. **Simplicity in Coordination**: By using natural language for agent communication, AutoGen eliminates the need for complex inter-agent protocols, reducing coordination complexity and making it easier to manage multiple agents.

3. **Scalability**: AutoGen is designed to handle multiple complex tasks simultaneously, allowing for efficient automation across various domains. This capability enhances overall scalability in your project.

4. **Customization and Flexibility**: The system offers fine-grained control and customization options, making it suitable for various use cases, whether simple or complex. This allows developers to tailor solutions to specific needs.

5. **Modularity and Maintainability**: The modular design of AutoGen makes it easy to maintain and extend. This advantage is particularly useful for developing sophisticated applications or for ongoing improvements and updates.

6. **Improved Factuality and Reasoning**: The structured collaboration between agents can enhance the factual accuracy of output and enable more sophisticated reasoning processes.

7. **Compatibility**: AutoGen works well with different Large Language Models (LLMs) and toolchains, providing flexibility for integration within existing systems.

These benefits make AutoGen a compelling choice for developing AI agents, especially in environments that require dynamic interactions and complex task management. 

TERMINATE

## Cons of AutoGen:
### Reasons in Favor of Choosing AutoGen:

1. **Scalability**: AutoGen's modular and extensible framework allows developers to create scalable and customizable AI systems easily, adapting to different project sizes and needs.

2. **Ease of Use**: The framework integrates observability and debugging tools which simplify the monitoring and control of agent workflows, making it more accessible for developers to manage AI interactions.

3. **Efficiency**: By reducing coordination complexity with natural language handoffs, AutoGen automates many mundane tasks, significantly lowering time-to-market for AI applications.

4. **Advanced Capabilities**: It supports complex automation and dynamic environments, making it suitable for sophisticated applications that require multiple AI agents to collaborate.

5. **Customization**: AutoGen allows high customization, enabling developers to tailor the AI solutions specifically to their project needs, enhancing the overall effectiveness.

### Cons of AutoGen:

1. **No Visual Flow Designer**: Unlike some competitors, AutoGen lacks a graphical interface for visualizing workflows, which could complicate debugging and system optimization, especially in complex multi-agent environments.

2. **Limitations in Creativity**: While it excels at generating outputs from predefined templates, AutoGen may not exhibit the same level of creativity or adaptability in dynamic scenarios compared to other frameworks.

3. **Learning Curve**: Users may find the learning curve steep, requiring significant time to master its complex features and functionalities, which could be a barrier for some developers.

4. **Cost of Multi-Agent Workflows**: Implementing multi-agent workflows can be expensive, which makes it difficult to justify their use in certain real-world applications.

5. **Integration Challenges**: For those looking for ease of integration and user-friendliness, AutoGen may not always meet expectations, potentially requiring more effort than alternative low-code solutions.

Based on these points, decision-makers should weigh the advantages of AutoGen against its limitations to determine the best fit for their AI Agent project. 

TERMINATE



## Decision:

Based on the research presented by your team, I recommend proceeding with AutoGen for the AI Agent project. The advantages of collaboration, efficiency, scalability, customization, and improved reasoning outweigh the drawbacks. While the lack of a visual flow designer and potential integration challenges are notable, the overall benefits—including simplified coordination and modularity—provide a strong foundation for developing dynamic and sophisticated AI applications. The ability to streamline workflows and support complex tasks aligns well with project goals, making AutoGen a compelling choice.

TERMINATE

In [21]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [22]:
await host.stop()